Connecting to a Neo4j Graph Database Using Python

In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  
username = "neo4j"             
password = "zjj20001126"          

driver = GraphDatabase.driver(uri, auth=(username, password))

def create_session():
    return driver.session()


Building a Knowledge Graph in Neo4j

In [2]:
import csv
def create_company(session, company_id, company_name, code):
    session.run("MERGE (:Company {id: $company_id, name: $company_name, code: $code})", 
                company_id=company_id, company_name=company_name, code=code)

def create_relationship(session, company1, relationship_type, company2):
    session.run(f"""
        MATCH (a:Company {{id: $company1}})
        MATCH (b:Company {{id: $company2}})
        MERGE (a)-[:{relationship_type}]->(b)
    """, company1=company1, company2=company2)

def load_data_and_create_graph():
    with driver.session() as session:
        with open('./KnowledgeGraph/hidy.nodes.company.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  
            for row in reader:
                create_company(session, row[0], row[1], row[2])

        # Read and create various types of relationships
        relationship_files = {
            'compete': './KnowledgeGraph/hidy.relationships.compete.csv',
            'cooperate': './KnowledgeGraph/hidy.relationships.cooperate.csv',
            'dispute': './KnowledgeGraph/hidy.relationships.dispute.csv',
            'invest': './KnowledgeGraph/hidy.relationships.invest.csv',
            'same_industry': './KnowledgeGraph/hidy.relationships.same_industry.csv',
            'supply': './KnowledgeGraph/hidy.relationships.supply.csv'
        }

        for rel_type, file_path in relationship_files.items():
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.reader(file)
                next(reader)  
                for row in reader:
                    create_relationship(session, row[0], rel_type, row[1])

load_data_and_create_graph()

In [3]:
# MATCH (c:Company {name: '杭汽轮B'})-[r]-(related:Company)
# RETURN related.name, type(r) as relationship_type


#### Analyzing Implicit Company Impacts Based on News Sentiment and Relationships in Neo4j
Classification by relationship type and news label: Based on the relationship type (same or opposite) and the positive or negative label of the news (1 or 0), the function determines whether these indirectly related companies are added to the positive or negative related set. For example, if the news is positive for a known company, then the company that competes with it (the opposite relationship) is seen as negatively related, and vice versa.

In [4]:
def find_implicit_companies(session, explicit_company_ids, news_label):
    implicit_positive = set()
    implicit_negative = set()

    # Query according to different relationship types
    relations = {
        'compete': 'opposite',
        'cooperate': 'same',
        'dispute': 'opposite',
        'invest': 'same',
        'same_industry': 'same',
        'supply': 'same'
    }

    for company_id in explicit_company_ids:
        for rel_type, impact_type in relations.items():
            query = f"""
            MATCH (c:Company {{id: $company_id}})-[:{rel_type}]-(implicit:Company)
            RETURN implicit.name AS company_name
            """
            result = session.run(query, company_id=company_id.strip())
            for record in result:
                implicit_company_name = record["company_name"]
                
                # Decide which collection to add based on news_label (0 or 1) and relational influence type
                if impact_type == 'same':
                    if news_label == 1:  # 1 positive
                        implicit_positive.add(implicit_company_name)
                    elif news_label == 0:  # 0 negative
                        implicit_negative.add(implicit_company_name)
                elif impact_type == 'opposite':
                    # Positive news is negative for the other side of the competition
                    if news_label == 1:  
                        implicit_negative.add(implicit_company_name)
                    # Negative news is positive for the other side of the competition
                    elif news_label == 0:  
                        implicit_positive.add(implicit_company_name)

    return list(implicit_positive), list(implicit_negative)





In [5]:
def get_company_id_by_name(session, company_name):
    query = "MATCH (c:Company {name: $company_name}) RETURN c.id AS company_id"
    result = session.run(query, company_name=company_name)
    return result.single()[0] if result.peek() else None


Test and check

In [9]:
from neo4j import GraphDatabase


uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(username, password))


company_names_to_query = "建设银行"
news_label = 0  # 或者 0

with driver.session() as session:
    # Split the company name and get the ID of each company
    company_ids = [get_company_id_by_name(session, name.strip()) for name in company_names_to_query.split(',')]

    company_ids = [company_id for company_id in company_ids if company_id is not None]

    # 对每个公司ID应用 find_implicit_companies 函数
    all_positive_companies = set()
    all_negative_companies = set()
    for company_id in company_ids:
        positive_companies, negative_companies = find_implicit_companies(session, [company_id], news_label)
        all_positive_companies.update(positive_companies)
        all_negative_companies.update(negative_companies)

    print("positive:", len(all_positive_companies))
    print("negative:", len(all_negative_companies))

# 关闭驱动连接
driver.close()



positive: 2
negative: 58


Identify Implicit_Positive_Company and Implicit_Negative_Company

In [8]:
import pandas as pd
from neo4j import GraphDatabase
from tqdm import tqdm


uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(username, password))


task1_data = pd.read_excel('Task1.xlsx')
task1_data=task1_data.iloc[:100] 

task1_data['Implicit_Positive_Company'] = None
task1_data['Implicit_Negative_Company'] = None


with driver.session() as session:
    for index, row in tqdm(task1_data.iterrows(), total=task1_data.shape[0], desc="processing progress"):
        explicit_companies_names = [name.strip() for name in row['Explicit_Company'].split(',')]
        explicit_companies_ids = [get_company_id_by_name(session, name) for name in explicit_companies_names if name]

        explicit_companies_ids = [company_id for company_id in explicit_companies_ids if company_id is not None]

        news_label = row['label']

        positive_companies, negative_companies = find_implicit_companies(session, explicit_companies_ids, news_label)

        positive_companies_str = ','.join(str(id) for id in positive_companies) if positive_companies else None
        negative_companies_str = ','.join(str(id) for id in negative_companies) if negative_companies else None

        task1_data.at[index, 'Implicit_Positive_Company'] = positive_companies_str
        task1_data.at[index, 'Implicit_Negative_Company'] = negative_companies_str
        # print(f"row {index} - positive:", len(positive_companies))
        # print(f"row {index} - negative:", len(negative_companies))



# task1_data.to_excel('Task2.xlsx', index=False)

driver.close()




task1_data

processing progress:   8%|▊         | 8/100 [00:02<00:18,  4.93it/s]

Row 0: Positive IDs - ['任子行', '捷捷微电'], Negative IDs - ['招商证券', '首开股份', '中国石油', '浙商银行', '上海银行', '星网锐捷', '新北洋', '汇金科技', '海通证券', '中嘉博创', '中国石化', '平安银行', '南天信息', '民生银行', '交通银行', '怡亚通', '南京银行', '吴通控股', '特发信息', '证通电子', '比亚迪', '我爱我家', '广电网络', '农业银行', '东方雨虹', '天津港', '中国人寿', '中国船舶', '招商银行', '来伊份', '中信银行', '奥拓电子', '中兴通讯', '京东方', '光大银行', '聚龙股份', '航天信息', '中国建筑', '古鳌科技', '梦网集团', '太平洋证券', '陆家嘴', '广电运通', '新华联', '泰达股份', '晨鸣纸业', '移为通信', '工商银行', '兴业银行', '中国铝业', '贵州茅台', '佳都科技', '中国铁建', '科大讯飞', '中金公司', '银之杰', '邮储银行', '中国银行']
Row 1: Positive IDs - ['成都银行', '申万宏源', '长春高新', '上海银行', '新北洋', '优博讯', '汇金科技', '建设银行', '南天信息', '东方财富', '紫金银行', '交通银行', 'TCL集团', '招商蛇口', '中信证券', '中国电信', '宁德时代', '山东矿机', '浩云科技', '水晶光电', '国民技术', '金风科技', '中信建投', '创识科技', '中国船舶', '招商银行', '中信银行', '中国长城', '富森美', '光大银行', '京东方', '盐津铺子', '润建股份', '牧原股份', '古鳌科技', '中国神华', '广电运通', '神州信息', '中装建设', '金地集团', '中科曙光', '工商银行', '兴业银行', '神州数码', '中国国航', '银之杰', '邮储银行', '中国银行'], Negative IDs - ['ST云维']
Row 2: Positive IDs - ['中国电建', '中国卫通', '寒武纪', '中国交建', '春秋航空',

processing progress:  18%|█▊        | 18/100 [00:02<00:06, 12.53it/s]

Row 10: Positive IDs - ['东方航空', '康芝药业', '宝钢股份', '日海智能', '长春高新', '浙江富润', '海康威视', '江铃汽车', '华西股份', '白云机场', '雅戈尔', '恒实科技', '易华录', '亚玛顿', '皖天然气', '安恒信息', '万华化学', '纳尔股份', '万达电影', '中国联通', '大智慧', '万孚生物', '万东医疗', '天合光能', '广和通', '蠡湖股份', '四川美丰', '工业富联', '宝莫股份', '东方通', '航天信息', '倍杰特', '润建股份', '紫光国微', '南京熊猫', '华贸物流', '数据港', '隆基股份', '卫星石化', '荣盛石化', '中国国航', '国泰君安', '五粮液', '博睿数据', '石化油服', '华大基因', '同花顺', '埃斯顿', '万马科技', '新疆交建', '海通证券', '华丰股份', '建设银行', '石化机械', '有方科技', '游族网络', '万马股份', '宁德时代', '泰山石油', '渤海轮渡', '新国都', '人民网', '兆易创新', '烽火通信', '震有科技', '南方航空', '中环股份', '鸿达兴业', '博实股份', '中南建设', '得润电子', '中电兴发', '上海临港', '宜通世纪', '山东高速', '大华股份', '北部湾港', '中科曙光', '神州数码', '唐人神', '先河环保', '兴民智通', '京东方', '中国石油', '金域医学', '七一二', '航天科技', '中国一重', '际华集团', '张家界', '中国广核', '中国电信', '比亚迪', '智慧能源', '中信建投', '国元证券', '东软集团', '上海石化', '奥拓电子', '惠发食品', '东方日升', '吉大通信', '东方电气', '中兴通讯', '温氏股份', '森源电气', '美格智能', '飞利信', '天源迪科', '君实生物', '上海电力', '方大炭素', '深南电路', '工商银行', '东土科技', '兴业银行', '网宿科技', '会畅通讯', '立讯精密', '科大讯飞', '康希诺', '瑞丰新材', '迪普科技', '宝泰隆', '名家汇'

processing progress:  29%|██▉       | 29/100 [00:02<00:03, 22.61it/s]

...

processing progress:  94%|█████████▍| 94/100 [00:03<00:00, 78.33it/s]

Row 85: Positive IDs - ['中国石油', '方正证券', 'ST金贵', '生物股份', '大族激光', '中国石化', '智慧能源', '浙江医药', '东诚药业', '科伦药业', '唐人神', '金达威', '温氏股份', '高德红外', '牧原股份', '安迪苏'], Negative IDs - []
Row 86: Positive IDs - ['百隆东方', '联发股份'], Negative IDs - []
Row 87: Positive IDs - ['天味食品', '首旅酒店', '民德电子', '复旦张江', '浙商证券', '广汽集团', '安信信托', '通威股份', '中国联通', '辽宁成大', '新开普', '华友钴业', '光大银行', '浦发银行', '京北方', '新华联', '中国能建', '邮储银行', '金隅集团', '浙商银行', '京能电力', '海通证券', '建设银行', '中储股份', '新华网', '宏达股份', '青松建化', '我爱我家', '江苏银行', '中泰证券', '金风科技', '金新农', '金融街', '中国人寿', '财达证券', '聚龙股份', '中国平安', '长电科技', '江苏租赁', '山东高速', '复星医药', '山东黄金', '陆家嘴', '中国石油', '贵阳银行', '东方财富', '交通银行', '招商蛇口', '中国广核', '中国电信', '北京银行', '比亚迪', '农业银行', '中信建投', '雅本化学', '元隆雅图', '九州通', '中兴通讯', '四川路桥', '沃森生物', '恒生电子', '正元智慧', '工商银行', '兴业银行', '中国铁建', '迪普科技', '光大证券', '汇金科技', '中交地产', '招商公路', '平安银行', '伊利股份', '民生银行', '紫光股份', '中信证券', '美的集团', '新城控股', '上港集团', '浩云科技', '长沙银行', '中国太保', '招商银行', '中信银行', '上海电气', '耀皮玻璃', '中材国际', '越秀金控', '华能国际', '贵州茅台', '中国人保', '三一重工'], Negative IDs - ['海通证券', '广州发展

processing progress: 100%|██████████| 100/100 [00:03<00:00, 27.84it/s]

Row 98: Positive IDs - ['康芝药业', '海康威视', '创新医疗', '钧达股份', '方正证券', '亚玛顿', '红塔证券', '皖天然气', '浙商证券', '万华化学', '纳尔股份', '中国联通', '长江证券', '维宏股份', '天合光能', '四川美丰', '宝莫股份', '倍杰特', '隆基股份', '奥瑞金', '卫星石化', '中银证券', '荣盛石化', '中国国航', '国泰君安', '五粮液', '石化油服', '虹软科技', '安琪酵母', '埃斯顿', '中国电影', '新疆交建', '海通证券', '建设银行', '石化机械', '长城证券', '万马股份', '宁德时代', '泰山石油', '新希望', '中泰证券', '中环股份', '鸿达兴业', '乐普医疗', '亿嘉和', '创维数字', '博实股份', '中南建设', '中国中冶', '北部湾港', '山东高速', '中科曙光', '华西证券', '福莱特', '西部超导', '中金公司', '招商证券', '中国石油', '七一二', '航天科技', '小康股份', '中国一重', '际华集团', '中国电信', '智慧能源', '比亚迪', '中国医药', '农业银行', '安科生物', '巨星科技', '上海石化', '惠发食品', '东方日升', '金证股份', '熊猫乳品', '中兴通讯', '铂力特', '森源电气', '东方证券', '飞利信', '上海电力', '方大炭素', '工商银行', '宁波银行', '瑞丰新材', '康希诺', '宝泰隆', '光大证券', '南京证券', '申万宏源', '海天味业', '华泰证券', '兰石重装', '平安银行', '中原证券', '东兴证券', '紫光股份', '益生股份', '福田汽车', '中信证券', '国新能源', '中国银河', '京沪高铁', '奇安信', '德赛西威', '东北证券', '中远海控', '天风证券', '天津港', '中国船舶', '招商银行', '财通证券', '上海电气', '奥美医疗', '国信证券', '西部证券', '兴业证券', '辉隆股份', '福斯特', '海陆重工', '广发证券', '新和成', '今天国际', '中国人保', '歌

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，...,"中国建设银行股份有限公司, 建设银行",0,"任子行,捷捷微电","招商证券,首开股份,中国石油,浙商银行,上海银行,星网锐捷,新北洋,汇金科技,海通证券,中嘉..."
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上表示...,"中国农业银行, 农业银行",1,"成都银行,申万宏源,长春高新,上海银行,新北洋,优博讯,汇金科技,建设银行,南天信息,东方财...",ST云维
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重...,"中国国航, 外运发展",1,"中国电建,中国卫通,寒武纪,中国交建,春秋航空,中国石化,山航B,工商银行,中国外运,农业银...",None
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左...,胜利股份,1,"新疆浩源,特锐德",None
4,5,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点，曾...,"中国银行, 中国交通银行, 中国建设银行, 中国移动",1,"天味食品,首旅酒店,民德电子,复旦张江,浙商证券,广汽集团,安信信托,通威股份,中国联通,辽...","海通证券,广州发展,民生银行,福耀玻璃,郑州银行,交通银行,泰禾集团,中金岭南,中信国安,包..."
...,...,...,...,...,...,...
95,198,沪深股市放量大升，而个股普涨，赚钱效应有扩散迹象。盘面情况看，有以下一些市场热点： ...,"贵州茅台, 柳工, 大庆华科, 张裕A, 宝钢股份, 古越龙山, 万科A, 中国石化",1,"康芝药业,退市海润,青岛啤酒,海康威视,贵州燃气,迈瑞医疗,泰禾集团,南山控股,方正证券,绿...","招商证券,贵绳股份,中国石油,山西汾酒,宁德时代,国元证券,中国平安,中信证券,辉丰股份"
96,200,□本报记者杨鉴旻段铸 近日，中国证监会发布公告称，１２月４日将对广深铁路股份有限公...,"大秦铁路, 广深铁路股份有限公司, 广深铁路",1,"宝塔实业,盈峰环境,唐山港,兴业证券,桐昆股份",None
97,201,□本报记者闫立良 昨日，上海汽车公告，其定向增发并整体上市一事及方案已获证监会核准...,"上海汽车集团股份有限公司, 上汽集团, 上海机场, 上海交运集团股份有限公司, 上海电气, ...",1,"东方航空,江铃汽车,白云机场,北方国际,金智科技,上柴股份,东港股份,横河模具,中国核电,苏...","乐视网,中利集团,天沃科技"
98,202,气势如虹的股市依然在书写着冬天的奇迹。在成功突破2000点关键点位后，昨日股市更是一度突破了...,"中国石化, 中信建投",1,"康芝药业,海康威视,创新医疗,钧达股份,方正证券,亚玛顿,红塔证券,皖天然气,浙商证券,万华...","中国石油,新光光电,中信证券,天际股份"


If not, fill in 'None'

In [2]:
import pandas as pd
task2_data = pd.read_excel('Task2.xlsx')


task2_data['Implicit_Positive_Company'].fillna('None', inplace=True)
task2_data['Implicit_Negative_Company'].fillna('None', inplace=True)


task2_data.to_excel('Task2.xlsx', index=False)
task2_data

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，...,建设银行,0,"捷捷微电,任子行","中金公司,东方雨虹,移为通信,邮储银行,招商证券,梦网集团,新华联,陆家嘴,招商银行,星网锐..."
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上表示...,农业银行,1,"中国国航,邮储银行,长春高新,水晶光电,山东矿机,招商银行,中信建投,光大银行,上海银行,盐...",ST云维
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重...,"中国国航, 外运发展",1,"中国石化,南方航空,中国交建,吉祥航空,中国外运,工商银行,中国卫通,春秋航空,中国电建,农...",None
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左...,胜利股份,1,"特锐德,新疆浩源",None
4,5,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点，曾...,中国银行,1,"中国太保,首旅酒店,金新农,聚龙股份,光大银行,京北方,江苏租赁,新城控股,上海电气,中信银...","福耀玻璃,泰禾集团,平煤股份,中银证券,招商银行,包钢股份,民生银行,中国平安,建投能源,紫..."
...,...,...,...,...,...,...
489884,1037031,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,亿华通,0,None,"中国船舶,东旭光电,仕佳光子,飞龙股份,宝泰隆,福田汽车,东风汽车,百奥泰"
489885,1037032,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,"依米康, 中泰证券",1,"中金公司,温氏股份,东北证券,国泰君安,乐歌股份,招商证券,永茂泰,中信建投,中国平安,国新...","华谊嘉信,西水股份"
489886,1037033,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,"中核科技, 天风证券",1,"中金公司,华泰证券,复旦微电,立讯精密,财达证券,东北证券,闻泰科技,国泰君安,招商银行,中...","三特索道,吉翔股份,中源家居"
489887,1037034,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,海特生物,1,"海尔生物,药明康德",None
